# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import csv
import os
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city.title())

# Print the city count to confirm sufficient count
cities = set(cities);
cities = list(cities);
print(cities)
print(len(cities))

['Maun', 'Kocevje', 'Faanui', 'Divo', 'Changji', 'Bokspits', 'Miles City', 'Quang Ngai', 'Ilhabela', 'Encheng', 'Nanakuli', 'Bati', 'Leningradskiy', 'Tessalit', 'Salgueiro', 'Majene', 'Kamenka', 'Zaysan', 'Angoche', 'Haibowan', 'Qarqin', 'Gharghoda', 'Iqaluit', 'Kabansk', 'Cape Town', 'Namatanai', 'Ushuaia', 'Great Yarmouth', 'Barrow', 'Alexandria', 'Bredasdorp', 'Vila Velha', 'Ahipara', 'Hihifo', 'Hailar', 'Havoysund', 'Nueve De Julio', 'Dano', 'Arraial Do Cabo', 'Haapiti', 'Jamestown', 'Palu', 'Ambilobe', 'Itupiranga', 'Port Lincoln', 'Sao Felix Do Xingu', 'Yei', 'Great Bend', 'Mataura', 'Nara', 'Tasiilaq', 'Sawtell', 'Cooma', 'Honiara', 'Boende', 'Armacao Dos Buzios', 'Airai', 'Hamilton', 'Belushya Guba', 'Villarrobledo', 'Acari', 'Ames', 'Tautira', 'San Juan', 'Xining', 'Wucheng', 'Bur Gabo', 'Narsaq', 'Ostrovnoy', 'Waspan', 'Kyzyl-Suu', 'Price', 'Abha', 'Aranos', 'Aykhal', 'Xiangdong', 'Punta Arenas', 'Utiroa', 'Surab', 'Tulun', 'Shahrud', 'Saint-Leu', 'Piedade', 'Ewa Beach', 'Geo

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).

In [23]:
cities_df = pd.DataFrame({
    "City": cities,
    'Latitude': '',
    'Longitude': '',
    'Temperature': '',
    'Humidity': '',
    'Cloudiness': '',
    'Wind Speed': '',
})

print(cities_df.shape)
cities_df.head()


(594, 7)


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Maun,,,,,,
1,Kocevje,,,,,,
2,Faanui,,,,,,
3,Divo,,,,,,
4,Changji,,,,,,


In [24]:
ow_query_url = "http://api.openweathermap.org/data/2.5/weather?"
params = {
    'appid': weather_api_key,
}



for (idx, row) in cities_df.iterrows():
    city = row['City']
    params['q'] = city
    ow_city_response = requests.get(ow_query_url, params=params).json()
    try:
        print(f'Requesting data from City {city} and number {idx}')
        cities_df.loc[idx, 'Latitude'] = ow_city_response['coord']['lat'];
        cities_df.loc[idx, 'Longitude'] = ow_city_response['coord']['lon'];
        cities_df.loc[idx, 'Temperature'] = ow_city_response['main']['temp'];
        cities_df.loc[idx, 'Humidity'] = ow_city_response['main']['humidity'];
        cities_df.loc[idx, 'Cloudiness'] = ow_city_response['clouds']['all'];
        cities_df.loc[idx, 'Wind Speed'] = ow_city_response['wind']['speed'];
    except:
        print(f'Requesting data from City {city} and number {idx} failed')
    print('--------------------------------------------------------------')


Requesting data from City Maun and number 0
--------------------------------------------------------------
Requesting data from City Kocevje and number 1
--------------------------------------------------------------
Requesting data from City Faanui and number 2
--------------------------------------------------------------
Requesting data from City Divo and number 3
--------------------------------------------------------------
Requesting data from City Changji and number 4
--------------------------------------------------------------
Requesting data from City Bokspits and number 5
Requesting data from City Bokspits and number 5 failed
--------------------------------------------------------------
Requesting data from City Miles City and number 6
--------------------------------------------------------------
Requesting data from City Quang Ngai and number 7
--------------------------------------------------------------
Requesting data from City Ilhabela and number 8
-----------------

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [31]:
cities_df = cities_df[cities_df['Latitude'] != '']
print(cities_df.count)
cities_df.to_csv('Cities.csv')
cities_df.head(20)





<bound method DataFrame.count of             City Latitude Longitude Temperature Humidity Cloudiness Wind Speed
0           Maun   -19.98     23.42      296.15       38          0        2.6
1        Kocevje    45.64     14.86      283.92       84         39       0.89
2         Faanui   -16.48   -151.75      300.61       80         42       6.93
3           Divo     5.84     -5.36       300.2       75          0       3.12
4        Changji    44.02     87.32      291.15       27        100          2
..           ...      ...       ...         ...      ...        ...        ...
589  Clyde River    70.47    -68.59      262.15       78         75        7.7
590       Xinzhi     36.5     111.7      297.84       14          9       1.77
591        Kagay     6.01    120.91      299.78       81         44       1.06
592          Buy    58.48     41.53      277.65       72        100       2.39
593       Laguna    38.42   -121.42      294.71       52          1        2.1

[546 rows x 7 colu

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Maun,-19.98,23.42,296.15,38,0,2.6
1,Kocevje,45.64,14.86,283.92,84,39,0.89
2,Faanui,-16.48,-151.75,300.61,80,42,6.93
3,Divo,5.84,-5.36,300.2,75,0,3.12
4,Changji,44.02,87.32,291.15,27,100,2
6,Miles City,46.41,-105.84,291.15,20,1,0.84
7,Quang Ngai,15.12,108.8,296.38,89,0,1.91
8,Ilhabela,-23.78,-45.36,295.66,85,49,3.76
9,Encheng,22.18,112.3,295.94,92,3,1.79
10,Nanakuli,21.39,-158.15,297.45,69,90,2.6


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots


In [ ]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression